Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import st_loss
import time
import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils

import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
category = "real_data"
files = glob2.glob(f'{path}{category}/*.h5')
files = [f[len(f"'{path}{category}"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
optimal = pd.read_pickle(f"{path}output/pickle_results/real_data/optimal_input_size.pkl")
optimal.head()

,dataset,nb_genes
0,Quake_Smart-seq2_Trachea,500
1,Quake_Smart-seq2_Diaphragm,1000
2,Quake_10x_Spleen,500
3,Young,1000
4,mouse_ES_cell,1000


In [4]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [5]:
df = pd.DataFrame()
dropout = 0.9
lr = 0.4
layers = [200, 40, 60]
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")

    
    nb_genes = optimal[optimal["dataset"] == dataset]["nb_genes"].values[0]
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    cluster_number = np.unique(Y).shape[0]

    X = train.preprocess(X, nb_genes=nb_genes)
    for train_size in [ 1, 0.75, 0.5, 0.25
    ]:
        for run in range(3):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            if train_size == 1:
                X_train = X
                y_train = Y
            else:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, Y, train_size=train_size, random_state=run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            start = time.time()
            dresults = train.run(X_train,
                                 cluster_number,
                                 dataset,
                                 Y=y_train,
                                 nb_epochs=30,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 
                                 
                                 layers=layers,
                                 save_to=f"{path}output/{category}/{run}/",
                                 save_pred=False)
            elapsed = time.time() - start
            dresults["temperature"] = temperature
            dresults["dropout"] = dropout
            dresults["train_size"] = train_size
            dresults["layers"] = str(layers)
            dresults["run"] = run
            dresults["time"] = elapsed
            #         print(f".", end = "")
            #         print(f"# {temperature}, {dropout}, {lr}, {layers}",
            #               dresults.get('COMBINED_kmeans_ari', ""),
            #               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
            #               dresults.get('leiden_ari_0', ""))
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(
                f"{path}output/pickle_results/{category}/{category}_train_size.pkl"
            )

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8258284521205065
(1350, 23341) (1350, 23341) keeping 500 genes
>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1000 genes
>>>>> Data Quake_10x_Spleen
SCZI  0.9207243162607069
(9552, 23341) (9552, 23341) keeping 500 genes
>>>>> Data Young
SCZI  0.6966379900869195
(5685, 33658) (5685, 33658) keeping 1000 genes
>>>>> Data mouse_ES_cell
SCZI  0.8004502714264402
(2717, 24175) (2717, 24175) keeping 1000 genes
>>>>> Data Adam
SCZI  0.8632277666545504
(3660, 23797) (3660, 23797) keeping 500 genes
>>>>> Data Quake_10x_Bladder
SCZI  0.9828247498197693
(2500, 23341) (2500, 23341) keeping 500 genes
>>>>> Data Quake_Smart-seq2_Lung
SCZI  0.754207987444416
(1676, 23341) (1676, 23341) keeping 1000 genes
>>>>> Data Quake_10x_Limb_Muscle
SCZI  0.9609079013375856
(3909, 23341) (3909, 23341) keeping 500 genes
>>>>> Data worm_neuron_cell
SCZI  0.06355801295995496
(4186, 13488) (4186, 13488) keeping 5000 genes
>>>>

In [ ]:
X_train.shape

In [ ]:
df.groupby("train_size").mean()

In [ ]:
df.mean()#0.686020 20 # 0.631504:10 # 0.693487 : 30 #0.685507